<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/item-based-cf/item_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
import os
def is_running_on_colab():
    return "COLAB_GPU" in os.environ

LOCAL = not is_running_on_colab()

In [2]:
#@title Download necessary libraries
if not LOCAL:
    !pip install pyspark -qq
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

In [3]:
#@title Imports
import numpy as np
import json

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, VectorUDT

from tqdm.notebook import tqdm
import time
import gc

if not LOCAL:
    from google.colab import drive

from typing import Tuple, Dict, List
from functools import reduce

In [4]:
#@title Set up variables
if not LOCAL:
    JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
    GDRIVE_DIR = "/content/drive"
    GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
    GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
    AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
    LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
    MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
    SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
else:
    GDRIVE_DATA_DIR = os.path.abspath("./data")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "full_dataset")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    JAVA_HOME = "/opt/homebrew/opt/openjdk"
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [5]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

23/06/28 00:12:55 WARN Utils: Your hostname, MacBook-Air-di-Domiziano.local resolves to a loopback address: 127.0.0.1; using 192.168.1.175 instead (on interface en0)
23/06/28 00:12:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/28 00:12:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
if not LOCAL:
    drive.mount(GDRIVE_DIR, force_remount=True)

In [7]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x1539b9480>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.startTime', '1687903975567'),
  ('spark.app.name', 'PySparkTutorial'),


# Data acquisition

In [8]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])




In [9]:
# playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
# audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

# Item-Based Collaborative Filtering

Item-Based Collaboartive Filtering is the "transpose" approach to user-based CF. This time we won't consider the users' feature vectors, but the items'.
An item's rating vector $\mathbf{r}_i$ is the vector of ratings given to the item $i$ for all the users.

Let $m$ be the number of users, $n$ the number of playlists, then $\mathbf{r}_i \in \mathbb{R}^m$ and $\mathbf{R} \in \mathbb{R}^{m \times n}$.

In order to make a prediction, we take the set of items $I_u$ rated by the user $u$, and we compute the set $I^k_u$ of the top-$k$ most similar items to $i$ rated by $u$, for each item $i \in I_u$. Once done that, we average the $k$ rating vectors weighting them by their respective similarity.

In [10]:
DEBUG = False

In [11]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-train-{NUM_PLAYLISTS}.json") #TODO: Little bug this is songs_df, meaning it hasn't got any info, but we don't actually care.
RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-{NUM_PLAYLISTS}.txt")
with open(RATING_VECTOR_LENGTH_PATH, "r") as f:
  RATING_VECTOR_LENGTH = int(f.read())

songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
songs_df = spark.read.json(SONGS_INFO_DF)

playlist_map_schema = StructType([
    StructField("track_uri", StringType(), True),
    StructField("embedding", VectorUDT(), True)
])
PLAYLIST_MAP_PATH = os.path.join(SAVED_DFS_PATH, f"playlist_map-{NUM_PLAYLISTS}.json")
playlist_map = spark.read.schema(playlist_map_schema).json(PLAYLIST_MAP_PATH)

In [12]:
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)

In [13]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

The similarity between each couple is intractable, we can cluster the similar tracks in buckets using Locally Sensitive Hashing.

In [14]:
from pyspark.ml.feature import MinHashLSH, MinHashLSHModel

NUM_HASH_TABLES = 10
LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"lsh_model-{NUM_HASH_TABLES}-{NUM_PLAYLISTS}.pickle")
if os.path.exists(LSH_MODEL_PATH):
  model = MinHashLSHModel.load(LSH_MODEL_PATH)
else:
  mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=NUM_HASH_TABLES)
  model = mh.fit(playlist_map)
  model.save(LSH_MODEL_PATH)

model.transform(playlist_map).show(truncate=False)

+------------------------------------+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|track_uri                           |embedding                                                                        |hashes                                                                                                                                                               |
+------------------------------------+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|spotify:track:001BVhvaZTf2icV88rU3DA|(100001,[7109,25065,41600,60248,61080,65885,70937],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|[[4.00933355E8], [4

Since the MinHasLSH model is not so fast, let's try with another type of LSH model

In [15]:
# from pyspark.ml.feature import BucketedRandomProjectionLSH, BucketedRandomProjectionLSHModel

# BRP_LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"brp_lsh_model-{NUM_PLAYLISTS}.pickle")
# if os.path.exists(BRP_LSH_MODEL_PATH):
#   model = BucketedRandomProjectionLSHModel.load(BRP_LSH_MODEL_PATH)
# else:
#   brp = BucketedRandomProjectionLSH(inputCol="embedding", outputCol="hashes", numHashTables=5, bucketLength=1.0)
#   model = brp.fit(playlist_map)
#   model.save(BRP_LSH_MODEL_PATH)

# model.transform(playlist_map).show()

In [16]:
# #TODO: Remove this once tested
# def transform_to_aggregate(df: DataFrame) -> DataFrame:
#   """
#   Transforms the dataframe in order to be compatible for the union with an aggregate df.
#   """
#   return df.withColumn("sum", F.col("distCol")).withColumnRenamed("distCol", "squared_sum")

# def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
#   """
#   Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
#   with a simple df (track_uri, embedding, hashes, distCol).
#   """
#   other_df = transform_to_aggregate(other_df)
#   merged_df = aggregate_df.unionAll(other_df)
#   return merged_df.groupBy("track_uri", "embedding", "hashes").agg(
#     F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
#     F.sum("sum").alias("sum"))

# if DEBUG:
#   key = playlist_map.first()[1]
#   neigh = model.approxNearestNeighbors(playlist_map, key, 10)
#   merged_df_2 = merge_aggregate_df(transform_to_aggregate(neigh), neigh)
#   merged_df_2.show() 

In [17]:
def df_to_dict(df: DataFrame) -> DataFrame:
  return df.select("track_uri", "distCol").rdd.collectAsMap()

def merge_dicts(d1: Dict[str, float], d2: Dict[str, float]) -> Dict[str, float]:
  for key, value in d2.items():
    if key in d1:
      if type(d1[key]) is float:
        d1[key] = [d1[key]]
      d1[key] += [value]
    else:
      d1[key] = [value]
  return d1

if DEBUG:
  key = playlist_map.first()[1]
  neigh = model.approxNearestNeighbors(playlist_map, key, 10).cache()
  merged_df_2 = merge_dicts(df_to_dict(neigh), df_to_dict(neigh))
  print(merged_df_2)

In [18]:
def extract_similar_songs(playlist_tracks: DataFrame, playlist_map, model, k=10, disable_pbar=False) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_map.join(F.broadcast(playlist_tracks), "track_uri").select("track_uri", "embedding")
  transformed_tracks_embeddings = model.transform(tracks_embedding).cache()
  k_neighs = []

  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors', disable=disable_pbar):
    k_neigh = model.approxNearestNeighbors(playlist_map, row["embedding"], k).cache()
    k_neighs.append(df_to_dict(k_neigh))

  aggregate_df = reduce(merge_dicts, k_neighs)

  transformed_tracks_embeddings.unpersist()
  return aggregate_df

if DEBUG:
  # first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*").distinct()
  first_playlist = train_df.filter("pid == 1005").select(F.explode("tracks")).select("col.*").distinct()
  recommendations = extract_similar_songs(first_playlist, playlist_map, model, k=10)

We are safe by using python dictionaries since the data will be very small, and so dictionaries will be faster than pyspark dataframes.

In [19]:
if DEBUG:
  import sys
  print(f"The reccomendation dictionary is {sys.getsizeof(recommendations) / 1_000} KB")

In [20]:
def aggregate_recommendations(recommendations: Dict[str, float | List[float]]) -> DataFrame:
  aggregated = {}
  for key, value in recommendations.items():
    if type(value) is list:
      aggregated[key] = sum(x for x in value) / len(value)
    else:
      aggregated[key] = value
  # return {k: v for k, v in aggregated.items() if v != 0}
  # aggregated = sorted(aggregated.items(), key=lambda x: x[1])

  recommendations_schema = StructType([
      StructField("track_uri", StringType(), True),
      StructField("distance", FloatType(), True)
  ])


  recommendations_df = spark.createDataFrame(data=aggregated.items(), schema=recommendations_schema)

  return recommendations_df

if DEBUG:
  k = 40
  recommendations_df = aggregate_recommendations(recommendations)
  recommendations_df.show(truncate=False)
  first_playlist.show(truncate=False)
  print(recommendations_df.count(), first_playlist.count())

Now that we have a prediction, we can put the result in a pyspark dataframe in order to be used later

In [21]:
def remove_existing_tracks(playlist_tracks: DataFrame, recommendations_df: DataFrame) -> DataFrame:
  playlist_tracks = playlist_tracks.select("track_uri").cache()
  playlist_tracks_compatible = playlist_tracks.join(F.broadcast(recommendations_df), on="track_uri")
  playlist_tracks.unpersist()
  return recommendations_df.exceptAll(playlist_tracks_compatible)

if DEBUG:
  clean_df = remove_existing_tracks(first_playlist, recommendations_df)
  clean_df.show()

Putting all togheter:

In [22]:
def item_based_recommendation(playlist: DataFrame,playlist_map: DataFrame, model: MinHashLSHModel, k=50):
  playlist_songs = playlist.select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(playlist_songs, playlist_map, model, 20, disable_pbar=True)
  recommendations_df = aggregate_recommendations(recommendations)
  recommendations_df = remove_existing_tracks(playlist_songs, recommendations_df)
  return recommendations_df.orderBy(F.col("distance").asc()).limit(k).cache()

if DEBUG:
  playlist = train_df.filter("pid == 2005")
  result = item_based_recommendation(playlist, playlist_map, model)
  result.show()

# Performance Evaluation

In [23]:
def precision_at_k(recommendations, ground_truth, num_of_recommendations) -> float:
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").cache()
    reccomended_relevant_tracks_count = recommended_relevant_tracks.count() #this can be top_n_results.join in order to be more performant
    recommended_relevant_tracks.unpersist()
    precision = reccomended_relevant_tracks_count / float(num_of_recommendations)

    return precision


import math
#TODO: make it more efficient somehow
def normalized_discounted_cumulative_gain(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
  recommendations = recommendations.orderBy(F.col("distance").asc())
  recommendations_list = recommendations.collect()
  cumulative_gain = 0

  intersection = recommendations.join(ground_truth, "track_uri").count()
  if intersection == 0: return 0

  ideal_cumulative_gain = 1 + sum((1 / math.log(i, 2)) for i in range(2, 2+intersection))
  for index, row in enumerate(recommendations_list):
    i = index + 1
    is_rel = ground_truth.filter(F.col("track_uri").isin(row.track_uri)).count() > 0
    rel = 1 if is_rel else 0
    if i == 1:
      cumulative_gain += rel
    else:
      cumulative_gain += (rel / math.log(i, 2))
  return cumulative_gain / ideal_cumulative_gain

Executing performance evaluation on a random sample of the test set

In [24]:
def evaluate(pid: int, playlist_map) -> Tuple[DataFrame, float]:
    t1 = time.time()

    playlist_train = train_df.filter(f"pid == {pid}").cache()
    playlist_test = test_df.filter(f"pid == {pid}").cache()
    ground_truth = playlist_test.select(F.explode("tracks")).select("col.*").cache()
    num_of_recommendations = ground_truth.count()
    recommendations = item_based_recommendation(playlist_train, playlist_map, model, k=num_of_recommendations).cache()

    precision = precision_at_k(recommendations, ground_truth, num_of_recommendations)
    gain = normalized_discounted_cumulative_gain(recommendations, ground_truth, num_of_recommendations)

    t2 = time.time()
    print(f"Total time: {t2-t1}")

    playlist_train.unpersist()
    playlist_test.unpersist()
    ground_truth.unpersist()
    recommendations.unpersist()

    return playlist_train, playlist_test, ground_truth, recommendations, precision, gain


if DEBUG:
  train, test, gt, rec, prec, gain  = evaluate(1005, playlist_map)
  train.show(), test.show(), gt.show(), rec.show(truncate=False)
  print(f"Precision: {prec}, Gain: {gain}")

In [25]:
LAST_CHECKPOINT_INDEX = 240
EVALUATION_RESULTS_PATH = os.path.join(GDRIVE_DATA_DIR, "item_base_evaluation", "IB_evaluation_results_FINAL")

def perform_evaluation():
  SAMPLING_FRACTION = 0.01
  sampled_playlists = train_df.sample(False, SAMPLING_FRACTION, seed=42).cache()

  transformed_playlist_map = model.transform(playlist_map).cache()
  results = []
  for index, row in enumerate(tqdm(sampled_playlists.collect(), desc="Performing evaluation")):
      if index <= LAST_CHECKPOINT_INDEX: continue 

      CHECKPOINT_RESULTS = os.path.join(GDRIVE_DATA_DIR, "item_base_evaluation", f"IB_evaluation_results_check_{index}")
      pid = row['pid']
      train, test, gt, rec, prec, gain = evaluate(pid,transformed_playlist_map)
      print((prec, gain))
      results.append((prec, gain))
      if index % 10 == 0:
         with open(CHECKPOINT_RESULTS, "w") as f:
            json.dump(results, f)
  
  sampled_playlists.unpersist() 
  with open(EVALUATION_RESULTS_PATH, "w") as f:
    json.dump(results, f)
  return results

results = perform_evaluation()

Performing evaluation:   0%|          | 0/1024 [00:00<?, ?it/s]

23/06/28 00:13:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/06/28 00:15:46 WARN CacheManager: Asked to cache already cached data.        


Total time: 166.28023505210876
(0.07142857142857142, 0.43824088773922626)


23/06/28 00:16:50 WARN CacheManager: Asked to cache already cached data.        


Total time: 60.61270499229431
(0.0, 0)


...

Total time: 388.57246112823486
(0.2, 0.9020243604942442)


23/06/28 09:50:36 WARN CacheManager: Asked to cache already cached data.        


Total time: 169.05308175086975
(0.0, 0)


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 9]
Traceback (most recent call last):
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [29]:
LAST_CHECKPOINT_RESULTS = os.path.join(GDRIVE_DATA_DIR, "item_base_evaluation", f"IB_evaluation_results_check_240")
with open(LAST_CHECKPOINT_RESULTS, "r") as f:
    results = json.load(f)

avg_prec = np.array(results).mean()
avg_prec, avg_gain = 0, 0
for prec, gain in results:
  avg_prec += prec
  avg_gain += gain 
tot = len(results)
avg_prec /= tot
avg_gain /= tot
avg_prec, avg_gain

(0.09350849349560382, 0.26496025670885603)

In [35]:
results

[(0.23076923076923078, 0.8403030283801005),
 (0.16666666666666666, 0.25),
 (0.045454545454545456, 0.7601875334318686),
 (0.42857142857142855, 0.37488434580081953),
 (0.5, 0.5),
 (0.0, 0),
 (0.16071428571428573, 0.7963065271236838),
 (0.1, 0.16666666666666666)]

# All against all tries

Since computing the k nearest neighbors is super slow, I can pre-compute them offline and store them. This will require like 100 years lol.

In [36]:
mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
model = mh.fit(playlist_map)

In [ ]:
def compute_all_k_neighbors(playlist_map: DataFrame, model) -> DataFrame:
    result = []
    transformed_playlist_map = model.transform(playlist_map).cache()
    for index, row in enumerate(tqdm(playlist_map.collect(), desc="Computing k-neighbors")):
        k_neighs = model.approxNearestNeighbors(transformed_playlist_map, row.embedding, 10).select("track_uri", F.col("distCol").alias("similarity"))
        result.append((row.track_uri, k_neighs.collect()))

    k_neighs_schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("distCol", FloatType(), nullable=True)
    ])

    schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("k_neighs", ArrayType(k_neighs_schema), nullable=True)
    ])

    result_df = spark.createDataFrame(result, schema)
    transformed_playlist_map.unpersist()

    return result_df

result_df = compute_all_k_neighbors(playlist_map, model)

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Read timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedRead(NioSocketImpl.java:273)
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:299)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:340)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:789)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1025)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1019)
	at java.base/java.io.DataInputStream.readInt(DataInputStream.java:381)
	at org.apache.spark.security.SocketAuthHelper.readUtf8(SocketAuthHelper.scala:103)
	at org.apache.spark.security.SocketAuthHelper.authClient(SocketAuthHelper.scala:57)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:67)


In [ ]:
K_NEIGHBOURS_PATH = os.path.join(GDRIVE_DATA_DIR,"saved_models", f"k_neighbours-{NUM_PLAYLISTS}.parquet")
result_df.write.parquet(K_NEIGHBOURS_PATH)

# Old stuff

In [ ]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("similarity")).withColumnRenamed("similarity", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

In [ ]:
def create_similarity_df(input_vector: SparseVector, playlist_map: DataFrame, similarity_function: Callable) -> DataFrame:
  """
  Returns the similarity df for a single song
  """
  
  @F.udf(returnType=FloatType())
  def compute_similarity(vector1):
    return similarity_function(vector1, input_vector)

  result_df = playlist_map.withColumn("similarity", compute_similarity(playlist_map.embedding))
  
  return result_df

def get_top_k_results(track_uri: str, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter((F.col("similarity") > 0) & (F.col("track_uri") != track_uri)).orderBy(F.col("similarity").desc()).limit(k)

if DEBUG:
  first_song_vector = playlist_map.select("embedding").limit(1).first()[0]
  first_track_uri = playlist_map.select("track_uri").limit(1).first()[0]
  similarity_df = create_similarity_df(first_song_vector, playlist_map, jaccard_similarity)
  top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
  top_k_results.show()

In [ ]:
def extract_similar_songs(playlist_tracks: DataFrame, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")
  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = get_top_k_results(
        track_uri=row.track_uri,
        similarity_df= create_similarity_df(input_vector=row.embedding,
                                            playlist_map=playlist_map,
                                            similarity_function=jaccard_similarity),
        k = 10)
    
    k_neigh.show()
    if aggregate_df == None:
      aggregate_df = transform_to_aggregate(k_neigh).cache()
    else:
      aggregate_df = merge_aggregate_df(aggregate_df, k_neigh).cache()
    
    aggregate_df.show()
  return aggregate_df

if DEBUG:
  first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(first_playlist, model)

In [ ]:
recommendations.show()

In [ ]:
# def extract_similar_songs(playlist: DataFrame, playlist_map: DataFrame) -> Dict[str, float]:
#   """
#   For each track in the playlist, it extracts the top-k most similar results.
#   """
#   weighted_dict = {}
  
#   # F.udf(returnType=IntegerType())
#   def update_weighted_dict(track_uri: str, similarity: float):
#     nonlocal weighted_dict
#     if track_uri not in weighted_dict:
#       weighted_dict[track_uri] = (similarity, 1)
#     else:
#       curr_similarity = weighted_dict[track_uri][0]
#       count = weighted_dict[track_uri][1]
#       weighted_dict[track_uri] = (curr_similarity + similarity, count+1)
  
#   tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
#   tracks_mapped = tracks.join(playlist_map, "track_uri")

#   for track_row in tqdm(tracks_mapped.collect(), desc=f"Analyzing tracks"):
#     similarity_df = create_similarity_df(track_row.embedding, playlist_map, jaccard_similarity)
#     top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
#     top_k_results.foreach(lambda row: update_weighted_dict(row.track_uri, row.similarity))
#     top_k_results.unpersist()
  
#   return weighted_dict

# if DEBUG:
#   first_playlist = train_df.limit(1).cache()
#   weighted_dict = extract_similar_songs(first_playlist, playlist_map)

In [ ]:
def create_similarity_df(playlist: DataFrame, playlist_map: DataFrame, similarityFunction: Callable):
  # Get the current playlist's tracks
  playlist_tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
  playlist_tracks.show()
  # Let's extract the rating vector for each single track
  playlist_tracks_mapped = playlist_tracks.join(playlist_map, "track_uri")
  playlist_tracks_mapped.show()


  @F.udf(returnType=FloatType())
  def compute_similarity(input_vector: SparseVector, other_vector: SparseVector):
    return jaccard_similarity(input_vector, other_vector)

  # for row in playlist_tracks_mapped.collect():
  similarity_df = playlist_map.withColumn("similarity", compute_similarity(playlist_tracks_mapped.limit(2).embedding, playlist_map.embedding))
  # break
  
  similarity_df.show()
  return

first_playlsit = train_df.limit(1).cache()
create_similarity_df(first_playlsit, playlist_map, jaccard_similarity)

In [ ]:
first_playlsit.show()